# Code to generate Frustration Fouls and Euro Fouls

In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/"

league = "nba"
season_yr = "2020"
season_type = "Regular Season"
games_id = pbp_season(league=league,season_yr=season_yr,season_type=season_type)
games_list = pbp_games(games_id)

Number of games:  1080


100%|██████████| 1080/1080 [00:43<00:00, 24.74it/s]

Number of bad games:  0


In [2]:
from pbpstats.resources.enhanced_pbp import Foul
from pbpstats.resources.enhanced_pbp import Turnover
from pbpstats.resources.enhanced_pbp import FieldGoal
pos_store = []
TO_Miss_pID = []
Foul_pID = []
Foul_tID = []
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if isinstance(possession_event, Foul) and (isinstance(possession_event.previous_event, Turnover) or  (isinstance(possession_event.previous_event, FieldGoal) and not possession_event.previous_event.is_made)) and possession_event.seconds_since_previous_event <= 5:
                pos_store.append(possession)
                TO_Miss_pID.append(possession_event.previous_event.player1_id)
                # print ("Turnover/Missed Shot Player: {0}".format(possession_event.previous_event.player1_id))
                Foul_pID.append(possession_event.player1_id)
                Foul_tID.append(possession_event.team_id)
                # print ("Foul Player: {0}".format(possession_event.player1_id))

100%|██████████| 1080/1080 [00:00<00:00, 2300.75it/s]


In [3]:
year = int(season_yr)
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

In [4]:
Player_Name = []
a = Foul_pID
Foul_pID1, Fouls = np.unique(a, return_counts=True)  

for pID in Foul_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 

In [5]:

for i in range(len(Player_Name)):
    if not Player_Name[i]:
        Player_Name[i] =['abc']
Player_Name = list(itertools.chain(*Player_Name))
print(len(Player_Name))
print(len(Foul_pID1))

449
449


In [6]:
data_euroP = pd.DataFrame({'Foul_pID':Foul_pID1,'Player_Name':Player_Name,'Fouls':Fouls})
data_euroP = data_euroP[data_euroP['Player_Name'] != 'abc']
# data_euroP.to_csv ('Fast_Fouls_20_21_updated.csv',index=False)

In [7]:
Team_Name = [] 
a = Foul_tID
Foul_tID1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

In [8]:
np.sum(Fouls)

2591

In [9]:
data_euroT = pd.DataFrame({'Foul_tID':Foul_tID1,'Team_Name':Team_Name,'Fouls':Fouls})
# data_euroT.to_csv ('Fast_Fouls_20_21_teams_updated.csv',index=False)
data_euroT

,Foul_tID,Team_Name,Fouls
0,1610612737,Atlanta Hawks,108
1,1610612738,Boston Celtics,83
2,1610612739,Cleveland Cavaliers,81
3,1610612740,New Orleans Pelicans,65
4,1610612741,Chicago Bulls,93
5,1610612742,Dallas Mavericks,124
6,1610612743,Denver Nuggets,84
7,1610612744,Golden State Warriors,83
8,1610612745,Houston Rockets,101
9,1610612746,Los Angeles Clippers,105


In [10]:
FFoul_pID =[]
k = 0
for i in range(len(TO_Miss_pID)):
    if TO_Miss_pID[i] == Foul_pID[i]:
        FFoul_pID.append(Foul_pID[i])
        k +=1
        print(k)
        clear_output(wait=True)

725


In [11]:
Player_Name = []
a = FFoul_pID
FFoul_pID, Fouls = np.unique(a, return_counts=True)
for pID in FFoul_pID:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 

In [12]:
for i in range(len(Player_Name)):
    if not Player_Name[i]:
        Player_Name[i] =['abc']
Player_Name = list(itertools.chain(*Player_Name))
print(len(Player_Name))
print(len(FFoul_pID))

276
276


In [13]:
data_frust = pd.DataFrame({'FFoul_pID':FFoul_pID,'Player':Player_Name,'Fouls':Fouls})
data_frust = data_frust[data_frust['Player'] != 'abc']
# data_frust.to_csv ('Frustration_Fouls_20_21_updated.csv',index=False)

In [14]:
data_frust['rank'] = data_frust['Fouls'].rank(ascending=False) 
data_frust = data_frust.sort_values(by=['rank'])
data_frust_plot = data_frust.drop(columns=['FFoul_pID', 'rank'])
data_frust_plot.reset_index(drop=True,inplace=True)
data_frust_plot.reset_index(drop=False,inplace=True)
data_frust_plot["#"] = data_frust_plot["index"] +1
data_frust_plot = data_frust_plot[["#","Player","Fouls"]]
data_frust_plot = data_frust_plot.head(10)
data_frust_plot

,#,Player,Fouls
0,1,Chris Paul,15
1,2,Donovan Mitchell,13
2,3,Luka Doncic,12
3,4,Zach LaVine,10
4,5,Julius Randle,9
5,6,Nikola Jokic,9
6,7,Kyle Lowry,9
7,8,Russell Westbrook,9
8,9,Mike Conley,8
9,10,Trae Young,8


In [15]:
def plot_table(df,var,sort="Player",title=" ",col_width=15):
    fig = go.Figure(data=[go.Table(
        columnwidth=[5,40,col_width],
        header=dict(values=list(df.columns),
                    fill_color='blue',
                    align=['center','left','center'],
                    font=dict(color='snow',family="Arial Black, monospace", size=12),
                    line_color="snow"
                    ),
        cells=dict(values=[df["#"],df[sort],df[var]],
                fill_color='lavender',
                align=['center','left','center'],
                height=23,
                line_color="grey",#lightgrey",
                ),
                # height=25
        ),
    ])
    # fig.update_layout(title_text=title)
    fig.update_layout(title=dict(text=title,y=0.98,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
    tab_width = 250 + col_width
    tab_height = 310
    fig.update_layout(width=tab_width,height=tab_height,margin=dict(t=25,b=1,l=1,r=1))
    fig.add_annotation(x=0.0, y=0.0,text="@SravanNBA",showarrow=False,xshift=1,yshift=5)
    fig.add_annotation(x=1.0, y=0.0,text="Source: pbpstats",showarrow=False,xshift=1,yshift=5)
    # fig.update_layout(autosize=True)
    fig.show()
    # fig.write_image(fig_DIR+f"{var}.png", scale=2)
    return fig

In [16]:
fig = plot_table(data_frust_plot,"Fouls",title="Frustration Fouls 2022-23",col_width=15)
fig.write_image(fig_DIR+f"frust_foul.png", scale=2)

In [17]:
Team_Name = [] 
a = Foul_tID
Foul_tID1, Fouls = np.unique(a, return_counts=True)  
for tID in Foul_tID1:  
    Team_Name.append([team['full_name'] for team in team_dict if team['id'] == tID])
Team_Name = list(itertools.chain(*Team_Name))

data_euro = pd.DataFrame({'Foul_tID':Foul_tID1,'Team':Team_Name,'Fouls':Fouls})
data_euro['rank'] = data_euro['Fouls'].rank(ascending=False) 
data_euro = data_euro.sort_values(by=['rank'])
data_euro_plot = data_euro.drop(columns=['Foul_tID', 'rank'])
data_euro_plot.reset_index(drop=True,inplace=True)
data_euro_plot.reset_index(drop=False,inplace=True)
data_euro_plot["#"] = data_euro_plot["index"] +1
data_euro_plot = data_euro_plot[["#","Team","Fouls"]]
data_euro_plot = data_euro_plot.head(10)
data_euro_plot

,#,Team,Fouls
0,1,Utah Jazz,153
1,2,Dallas Mavericks,124
2,3,Charlotte Hornets,110
3,4,Atlanta Hawks,108
4,5,Los Angeles Clippers,105
5,6,Washington Wizards,104
6,7,Houston Rockets,101
7,8,New York Knicks,95
8,9,Chicago Bulls,93
9,10,Philadelphia 76ers,91


In [18]:
fig = plot_table(data_euro_plot,"Fouls",sort="Team",title="Euro Fouls 2022-23",col_width=15)
fig.write_image(fig_DIR+f"frust_foul.png", scale=2)